# <font color=black> Spinal cord networks: Functional connectivity </font>
<hr style="border:1px solid black">

### Imports

In [ ]:
import sys,json
import glob
import pandas as pd
import numpy as np
import nibabel as nib
import seaborn as sns
import os
import statsmodels.api as sm
from matplotlib import pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from sklearn.preprocessing import LabelEncoder
from statsmodels.stats.multitest import multipletests
from scipy.stats import spearmanr



main_dir='/cerebro/cerebro1/dataset/bmpd/derivatives/Aging_project/2025_brsc_aging_project/'
sys.path.append(main_dir+'/code/')
config_file=main_dir + '/config/analyses/brsc_functional.json'
config_file2=main_dir + '/config/preprocessing/01_brsc_preprocess_func.json'

from plotting import Plot_sc
from plotting import Plotting

import brsc_utils as utils
#from connectivity.post_icaps import post_icaps
from connectivity.seed_to_target import Seed2target
from brsc_statistics import Statistics 
%matplotlib inline
%load_ext autoreload
%autoreload 2

### load config file
with open(config_file) as config_f:
    config = json.load(config_f) # load config info

with open(config_file2) as config_f:
    config_proc = json.load(config_f) # load config info
stat_func=Statistics(config=config,ana_dir="",analysis="")

<hr style="border:1px solid black">

### <font color=#04AF98> A SPINAL CORD functional connectivity 
##### Initialize the analysis

In [ ]:
structure="spinalcord"


seed2seed=Seed2target(config_file,IDs=config["participants_IDs_ALL"],kind="corr",seed_kind="atlas",structure=[structure]) # initialize the function
ouputdir=config['project_dir'] + config["seed2seed"]['analysis_dir'][structure] 
plot=Plotting(config_file,ouputdir)


redo_indiv=True # rerun first level

#### <font color=#04AF98> B.2 Timeseries extraction

In [ ]:
timeseries,timeseries_labels,labels_list=seed2seed.extract_atlas_data(standardize='zscore',redo=False,n_jobs=3)

#### <font color=#04AF98> B.3 Correlation analysis

In [ ]:
#concatenante brain and spinal cord array
timeseries_zmean=[];labels_concat=[]

results = []
output_dir={};output_file={};

corr_concat_df,corr_half_concat_df,corr_mean_df =seed2seed.correlation_df(timeseries_labels,
                                                     timeseries_labels,
                                                    tag= "_C1C7_quad_z",
                                                         groups=["ALL",'YA','MA','OA'],
                                                         labels=True,
                                                      #partial_ts=timeseries_csf,
                                                         redo=False, n_jobs=1)



In [ ]:

output_dir= config["project_dir"] +"/figures/f02_functional_sc/revision_R1/seed2seed/spinalcord/"
colors = ["#F2ECDF","#D5E5D7","#7BCEBC","#40BFAA","#04AF98","#029E89","#008C79"]  # green

custom_cmap = LinearSegmentedColormap.from_list("my_colormap", colors)
for group in ["ALL"]:#,"YA","MA","OA"]:
    mean_matrix=corr_concat_df[group].groupby(["seed1", "seed2"], sort=False)[["fcorr"]].mean().reset_index()
    plot.plot_heatmap(matrix=corr_concat_df[group],
                  networks=labels_list,
                      vmax=0.25,vmin=0,
                      cmap=custom_cmap,
                      index_raw='seed1',index_col='seed2',metric='fcorr',
                      #labels=config["seeds"]["spinalcord"]["names"],
                  plot_networks=True,xlabels=True,ylabels=True,
                       output_f=output_dir + '/SpiFC_matrix.pdf',
                  #output_f=output_dir + '/figures/indiv/corr_matrix_' + ID + ".pdf",
                      save=False)
plt.show()

#### <font color=#04AF98> B.4 Intra vs Inter FC

In [ ]:
output_dir=config["project_dir"] +  "/figures/f02_functional_sc/preprint2025/seed2seed/spinalcord/"
from scipy import stats
mean_betwith=corr_half_concat_df['ALL'].groupby(["IDs","age","sex","betwith_labels"], sort=False)[["fcorr"]].mean().reset_index()
mean_betwith["betwith_labels"] = pd.Categorical(mean_betwith["betwith_labels"], categories=["intra", "inter"], ordered=True)
mean_betwith = mean_betwith.sort_values(by=["IDs", "age", "sex", "betwith_labels"]).reset_index(drop=True)


output_f=output_dir + '2_second_level/n67_betwith_fcorr.csv'
if not os.path.exists(output_f):
    mean_betwith.to_csv(output_f, index=False)


# ---- compute within versus between t-test
t_test=stats.ttest_rel(mean_betwith[mean_betwith["betwith_labels"]=="intra"]["fcorr"],mean_betwith[mean_betwith["betwith_labels"]=="inter"]["fcorr"])
print("t(" + str(t_test.df) + "): " + str(np.round(t_test.statistic,2)) + " p-value: " + str(t_test.pvalue))


plot.boxplots(df=mean_betwith,
              x_data="betwith_labels",x_order=["intra","inter"],
                  indiv_values=True,#invers_axes=True,
                  palette=["#04AF98","#B7DDCE"],#output_dir=config['main_dir'] + config['analysis_dir']['spinalcord'] + '/figures/',
                  #output_tag='corr_' + group_name,
              height=3,aspect=0.5,
                  y_data="fcorr",
              ymin=-0.1, ymax=0.5,
              output_dir=output_dir,
              output_tag="sim_intra-inter",
              save=False)
plt.show()

### <font color=#04AF98> Age effect

In [ ]:
output_dir_table=config["project_dir"] +  "/figures/f02_functional_sc/revision_R1/seed2seed/spinalcord/"

df=corr_half_concat_df["ALL"].groupby(["IDs","age","sex","betwith_labels"])["fcorr"].mean().reset_index()
df_intra=df[df["betwith_labels"]=="intra"]
df_inter=df[df["betwith_labels"]=="inter"]

if not os.path.exists(output_dir_table + "stats_sc_intra_FC.csv"):
    # Run bootstrap regression
    _, p_age,p_sex, beta_age,beta_sex, stat_age,stat_sex,_, ci_beta_age, ci_beta_cov = stat_func.regression_model(df=df_intra, y="fcorr", predictor="age", covariates=["sex"],random="IDs", n_bootstrap=1000)
    data = { "Predictor": ["age", "sex"],
            "β": [beta_age, beta_sex],
            "95% Bootstrap CI": [ci_beta_age, ci_beta_cov],
            "t-value": [stat_age, stat_sex],  # or t-value from results
            "p-value": [p_age, p_sex]}
    df_table = pd.DataFrame(data)
    df_table.to_csv(output_dir_table + "stats_sc_intra_FC.csv", index=False)
    
else:
    df_table = pd.read_csv(output_dir_table + "stats_sc_intra_FC.csv")
    
print(df_table)
plot.lmplots(df=df_intra,color=["#04AF98"],
                 x_data="age",
                 y_data="fcorr",xmin=15,xmax=85,
             indiv_values=True,
                 save=False)

if not os.path.exists(output_dir_table + "stats_sc_inter_FC.csv"):
    _, p_age,p_sex, beta_age,beta_sex, stat_age,stat_sex,_, ci_beta_age, ci_beta_cov=stat_func.regression_model(df=df_inter,y="fcorr",predictor="age",covariates=["sex"],random="IDs", n_bootstrap=1000)
    data = { "Predictor": ["age", "sex"],
                "β": [beta_age, beta_sex],
            "95% Bootstrap CI": [ci_beta_age, ci_beta_cov],
            "t-value": [stat_age, stat_sex],  # or t-value from results
            "p-value": [p_age, p_sex]}
    df_table = pd.DataFrame(data)
    df_table.to_csv(output_dir_table + "stats_sc_inter_FC.csv", index=False)
    
else:
    df_table = pd.read_csv(output_dir_table + "stats_sc_inter_FC.csv")

print(df_table)
plot.lmplots(df=df_inter,color=["#B7DDCE"],
                 x_data="age",
                 y_data="fcorr",xmin=15,xmax=85,
             indiv_values=True,
                 save=False)


In [ ]:
output_dir=config["main_dir"] +  "/figures/f02_functional_sc/seed2seed_nostd/spinalcord/"
#--- Global results
df_mean =corr_half_concat_df['ALL'].groupby(['IDs', 'group','age', 'sex'], as_index=False)['fcorr'].mean()
mean_results = [];
signed_r2_2, p_age,p_sex, beta_age,beta_sex, stat_age,stat_sex,_, ci_beta_age, ci_beta_sex=stat_func.regression_model(df=df_mean,y="fcorr",predictor="age",covariates=["sex"])
mean_results.append({"rois":"mean","signed_r2": signed_r2_2,"p_age": p_age,"p_sex": p_sex,"beta_age": beta_age,"beta_sex": beta_sex,"tvalue_age": stat_age,"tvalue_sex": stat_sex})
mean_result_df= pd.DataFrame(mean_results) 

#--- Results by regions
param={"intra":{"y_lim":[-2,3],"colors":["#e7cda4","#E2971F","#7BCEBC","#008C79","#40BFAA","#04AF98"]},
      "inter":{"y_lim":[0,4],"colors":["#C4EBE2","#A0DDCF","#40BFAA","#7BCEBC","#008C79","#04AF98"] }}

for region in ["inter","intra"]:
    print(region); roi_results=[];
    df_reduced=corr_half_concat_df['ALL'][corr_half_concat_df['ALL']["betwith_labels"]==region]
    roi_results=[];
    
    if not os.path.exists(output_dir_table + "stats_sc_"+region+"_FC_by_horn.csv"):
        for roi in ["dorsal","ventral","right","left","cross1","cross2"]:
            df_roi_metrics=df_reduced[(df_reduced["labels1"]==roi)].groupby(['IDs', 'age', 'sex',"seed1","seed2"], as_index=False)["fcorr"].mean()
            signed_r2_2, p_age,p_sex, beta_age,beta_sex, stat_age,stat_sex,_, ci_beta_age, ci_beta_sex=stat_func.regression_model(df=df_roi_metrics,y="fcorr",predictor="age",covariates=["sex"], random="IDs", n_bootstrap=1000)
            roi_results.append({"Region": region,"ROI": roi,"Predictor": "age","β": beta_age,"95% Bootstrap CI": ci_beta_age,"t-value": stat_age,"p-value": p_age,"p-fdr": p_age,})
            roi_results.append({"Region": region,"ROI": roi,"Predictor": "sex","β": beta_sex,"95% Bootstrap CI": ci_beta_sex,"t-value": stat_sex,"p-value": p_sex,"p-fdr": p_sex,})

        roi_results_df= pd.DataFrame(roi_results)
        # ---- FDR correction
        p_age = roi_results_df["p-value"][0]; roi_results_df["p-value"][1]
        reject, page_corrected, _, _ = multipletests(p_age, method='fdr_bh'); reject, psex_corrected, _, _ = multipletests(p_sex, method='fdr_bh')
        roi_results_df.loc[0, "p-fdr"] = page_corrected; roi_results_df.loc[1, "p-fdr"] = psex_corrected
        roi_results_df.to_csv(output_dir_table + "stats_sc_"+region+"_FC_by_horn.csv", index=False)
        
        
    else:
        roi_results_df = pd.read_csv(output_dir_table + "stats_sc_"+region+"_FC_by_horn.csv")
    
    sub_df=roi_results_df[roi_results_df["Predictor"]=="age"]
    plot.barplots(df=sub_df, x_data="ROI",x_order=["dorsal","ventral","right","left","cross1","cross2"], #x_order=["ventral","dorsal","within","cross"],
                      y_data="t-value",
                      palette=param[region]["colors"],
                      ymin=param[region]["y_lim"][0],ymax=param[region]["y_lim"][1],indiv_values=False,
                         output_dir=output_dir + "/figures/" ,
                      #height=3,aspect=0.5,
                      output_tag="age_effect_tvalues_quad",save=False)
    #plot.barplots(df=df, x_data="sub_metric", y_data="tvalue_sex",palette=palette,hue="contrast",ymin=-5,ymax=4,indiv_values=False)
    plt.show()
        
    print(roi_results_df)

In [ ]:
for region in ["intra"]:
    palette=["#E2971F","#E4A541","#E5B262","#e7cda4","#7BCEBC","#40BFAA","#04AF98","#008C79"]
     # Example data
    data = [-1.7,-0.1,1.4,1.5,2.3,2.7]
    
    fig, ax = plt.subplots(figsize=(len(palette), 1))
    for i, color in enumerate(palette):
        ax.add_patch(plt.Rectangle((i, 0), 1, 1, color=color))
    
    # Aesthetics
    ax.set_xlim(0, len(palette))
    ax.set_ylim(0, 1)
    ax.axis('off')
    plt.savefig(output_dir + '/figures/age_effect_tvalues_intraquad_colormap.pdf', format='pdf')
    plt.show()
